Importar librerías

In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, OrdinalEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import f1_score

Cargamos el dataset

In [52]:
df = pd.read_csv('./salario.csv')

Visualizamos el dataframe inicial

In [53]:
df

,edad,trabajo,estudios,estado-civil,trabajo.1,posicion-familiar,etnia,sexo,ganancias_inversiones,perdidas_inversiones,horas-trabajo_semana,pais-origen,salario
0,39,State-gov,Bachelors,Never-married,Adm-clerical,No-en-familia,Blanco,Hombre,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Marido,Blanco,Hombre,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,No-en-familia,Blanco,Hombre,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Marido,Negro,Hombre,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Mujer,Negro,Mujer,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27993,59,?,10th,Widowed,?,Soltero-a,Blanco,Mujer,0,0,40,United-States,<=50K
27994,42,Self-emp-not-inc,Masters,Married-civ-spouse,Exec-managerial,Marido,Blanco,Hombre,0,0,50,United-States,>50K
27995,62,Private,HS-grad,Widowed,Sales,No-en-familia,Blanco,Mujer,0,0,43,United-States,<=50K
27996,54,Private,HS-grad,Never-married,Sales,No-en-familia,Blanco,Hombre,0,0,40,United-States,<=50K


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27998 entries, 0 to 27997
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   edad                   27998 non-null  int64 
 1   trabajo                27998 non-null  object
 2   estudios               27998 non-null  object
 3   estado-civil           27998 non-null  object
 4   trabajo.1              27998 non-null  object
 5   posicion-familiar      27998 non-null  object
 6   etnia                  27998 non-null  object
 7   sexo                   27998 non-null  object
 8   ganancias_inversiones  27998 non-null  int64 
 9   perdidas_inversiones   27998 non-null  int64 
 10  horas-trabajo_semana   27998 non-null  int64 
 11  pais-origen            27998 non-null  object
 12  salario                27998 non-null  object
dtypes: int64(4), object(9)
memory usage: 2.8+ MB


Vamos a ver posibles valores nulos

In [55]:
for i in df.columns:
    print(df[i].value_counts().sort_values(ascending=False))

edad
31    764
23    763
36    762
35    756
34    754
     ... 
84     10
83      6
88      3
85      2
86      1
Name: count, Length: 72, dtype: int64
trabajo
Private             19483
Self-emp-not-inc     2211
Local-gov            1810
?                    1568
State-gov            1119
Self-emp-inc          961
Federal-gov           830
Without-pay            11
Never-worked            5
Name: count, dtype: int64
estudios
HS-grad         9033
Some-college    6250
Bachelors       4642
Masters         1477
Assoc-voc       1184
11th            1023
Assoc-acdm       905
10th             811
7th-8th          563
Prof-school      488
9th              438
12th             369
Doctorate        357
5th-6th          282
1st-4th          131
Preschool         45
Name: count, dtype: int64
estado-civil
Married-civ-spouse       12834
Never-married             9172
Divorced                  3837
Separated                  899
Widowed                    869
Married-spouse-absent      366
Married-A

Vemos que hay 3 columnas (trabajo, trabajo.1 y pais-origen) que tienen '?' como uno de los valores más repetidos. Vamos a cambiar este valor a valores nulos para poder tratarlos.  
Para ello, volvemos a cargar el dataset de la siguiente manera:

In [56]:
df = pd.read_csv('./salario.csv', na_values=['?'], skipinitialspace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27998 entries, 0 to 27997
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   edad                   27998 non-null  int64 
 1   trabajo                26430 non-null  object
 2   estudios               27998 non-null  object
 3   estado-civil           27998 non-null  object
 4   trabajo.1              26425 non-null  object
 5   posicion-familiar      27998 non-null  object
 6   etnia                  27998 non-null  object
 7   sexo                   27998 non-null  object
 8   ganancias_inversiones  27998 non-null  int64 
 9   perdidas_inversiones   27998 non-null  int64 
 10  horas-trabajo_semana   27998 non-null  int64 
 11  pais-origen            27503 non-null  object
 12  salario                27998 non-null  object
dtypes: int64(4), object(9)
memory usage: 2.8+ MB


Para evitar data leakage durante el preprocesado, haremos el train-validation split ahora.  
También, mapeamos el target para trabajar con 0 y 1

In [57]:
mapa_salario = {
    '<=50K': 0,
    '>50K': 1
}

df['salario'] = df['salario'].map(mapa_salario)


target_col = 'salario'
y = df[target_col]
X = df.drop(target_col, axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    stratify=y, 
    random_state=42
)

Comprobamos el split

In [58]:
print(f"Tamaño de X_train: {X_train.shape}")
print(f"Tamaño de X_val: {X_val.shape}")
print("\nProporción en y_train:")
print(y_train.value_counts(normalize=True))
print("\nProporción en y_val:")
print(y_val.value_counts(normalize=True))

Tamaño de X_train: (22398, 12)
Tamaño de X_val: (5600, 12)

Proporción en y_train:
salario
0    0.760425
1    0.239575
Name: proportion, dtype: float64

Proporción en y_val:
salario
0    0.760357
1    0.239643
Name: proportion, dtype: float64


Para los nulos, decidimos imputar valores ya que hay una cantidad de nulos considerables y si eliminaramos tanto filas como columnas estaríamos perdiendo información valiosa.  
Para ello, veamos que columnas tienen valores nulos

In [59]:
print(X_train.isnull().sum())

edad                        0
trabajo                  1249
estudios                    0
estado-civil                0
trabajo.1                1253
posicion-familiar           0
etnia                       0
sexo                        0
ganancias_inversiones       0
perdidas_inversiones        0
horas-trabajo_semana        0
pais-origen               396
dtype: int64


Imputamos con la moda al ser columnas de tipo 'object'

In [60]:
columnas_con_nulos = ['trabajo', 'trabajo.1', 'pais-origen']

for col in columnas_con_nulos:
    moda = X_train[col].mode()[0]
    X_train[col] = X_train[col].fillna(moda)
    X_val[col] = X_val[col].fillna(moda)

Comprobamos que se han imputado correctamente

In [61]:
print("Nulos restantes en X_train:")
print(X_train.isnull().sum())

Nulos restantes en X_train:
edad                     0
trabajo                  0
estudios                 0
estado-civil             0
trabajo.1                0
posicion-familiar        0
etnia                    0
sexo                     0
ganancias_inversiones    0
perdidas_inversiones     0
horas-trabajo_semana     0
pais-origen              0
dtype: int64


Ahora vamos a tratar las columnas para poder entrenar el modelo:

Para las columnas categóricas, hemos identificado tres tipos distintos de variables categóricas en nuestro dataset y, para maximizar la precisión del modelo, aplicaremos una estrategia de codificación específica para cada una:

- Variables Ordinales (con orden): La columna estudios tiene una jerarquía clara (ej. HS-grad < Bachelors < Doctorate). Para conservar esta valiosa información, usaremos un OrdinalEncoder con un mapa de orden definido, convirtiéndola en una única columna numérica (ej. 0 a 15).

- Variables Binarias (dos valores): La columna sexo solo tiene dos valores. Usar OneHotEncoder sería ineficiente. Aplicaremos un OrdinalEncoder (o .map()) para convertirla en una sola columna (0 o 1).

- Variables Nominales (sin orden): El resto de columnas como trabajo, estado-civil y pais-origen son etiquetas sin orden lógico. Para estas, usaremos OneHotEncoder, que es la única forma correcta de evitar que el modelo aprenda patrones falsos (ej. creer que "Mexico" > "Cuba").

En cuanto a las columnas numericas:

- Hay columnas como ganancias_inversiones, con muchísimos ceros y unos pocos valores altísimos (como 99999).
    No sabemos si ese 99999 es un error o si es información valiosa (ej. un código para "ganancias máximas").

    - Opción 1: Borrar outliers y usar StandardScaler
        Encontrar el 99999, borrarlo o cambiarlo por un valor "normal", y luego usar StandardScaler (que usa la media).
        El Riesgo: Si 99999 era un dato valioso, acabamos de destruir la información más importante de esa columna. La media ahora será incorrecta.

    - Opción 2: Usar RobustScaler (La que elegimos)
        Qué es: RobustScaler usa la mediana para escalar, no la media.
        La Ventaja: La mediana de nuestra columna es 0. Al RobustScaler no le importa el valor 99999 para calcular la escala, así que no se "contamina".

    Si 99999 era un error: RobustScaler lo ignoró.  
    Si 99999 era valioso: RobustScaler lo mantiene como un valor muy alto y único. El modelo aún puede verlo y aprender de él. ¡Perfecto!

In [62]:
numeric_features = ['edad', 'ganancias_inversiones', 'perdidas_inversiones', 'horas-trabajo_semana']
ordinal_features = ['estudios']
binary_features = ['sexo']
nominal_features = ['trabajo', 'estado-civil', 'trabajo.1', 'posicion-familiar', 'etnia', 'pais-origen']
education_order = [
    'Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', 
    '10th', '11th', '12th', 'HS-grad', 'Some-college', 
    'Assoc-voc', 'Assoc-acdm', 'Bachelors', 'Masters', 
    'Prof-school', 'Doctorate'
]

numeric_transformer = RobustScaler()
ordinal_transformer = OrdinalEncoder(
    categories=[education_order],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)
binary_transformer = OrdinalEncoder()
nominal_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('bin', binary_transformer, binary_features),
        ('nom', nominal_transformer, nominal_features)
    ],
    remainder='passthrough'
)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

print(f"Dimensiones (originales): {X_train.shape}")
print(f"Nuevas dimensiones (preprocesadas): {X_train_preprocessed.shape}")

Dimensiones (originales): (22398, 12)
Nuevas dimensiones (preprocesadas): (22398, 87)


Como tenemos datos desbalanceados, elegimos SMOTE porque es la mejor manera de equilibrar los datos sin perjudicar al modelo.  
Las otras opciones eran peores: 
- Undersampling (borrar datos) significaba perder información valiosa.  
- Random Oversampling (copiar datos) provoca que el modelo memorice y se sobreajuste (overfitting).
- SMOTE es la mejor solución porque, en lugar de copiar, crea nuevos datos sintéticos que son parecidos a los originales. Esto nos permite balancear el dataset para que el modelo aprenda mejor, pero sin perder información ni caer en el sobreajuste.

In [63]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(
    X_train_preprocessed, 
    y_train
)


print(f"Dimensiones de X_train (antes de SMOTE): {X_train_preprocessed.shape}")
print(f"Dimensiones de X_train (después de SMOTE): {X_train_balanced.shape}")

print("\nBalance de clases (antes de SMOTE):")
print(y_train.value_counts())

print("\nBalance de clases (después de SMOTE):")
print(y_train_balanced.value_counts())

Dimensiones de X_train (antes de SMOTE): (22398, 87)
Dimensiones de X_train (después de SMOTE): (34064, 87)

Balance de clases (antes de SMOTE):
salario
0    17032
1     5366
Name: count, dtype: int64

Balance de clases (después de SMOTE):
salario
0    17032
1    17032
Name: count, dtype: int64


Pasemos con la selección de características:


Métodos de filtro:
- Hemos elegido f_classif (f-score) como nuestro método de filtro porque se alinea perfectamente con las recomendaciones de la asignatura para nuestro problema.  
- En los apuntes se indica que el f-score es ideal para problemas de clase binaria, que es exactamente nuestro caso (salario >50K o <=50K).  
- Dado que nuestro preprocesado ha convertido las 103 características en datos numéricos, f_classif es la herramienta estadística idónea para puntuar cada característica midiendo la diferencia de sus medias entre las dos clases de salario.  
- Esto lo hace más apropiado que el Chi-cuadrado, que es para datos categóricos , o la correlación de Pearson, que solo mide relaciones lineales. 

- La selección del número de características, $k$, es en sí misma un hiperparámetro. Para realizar la evaluación comparativa de los tres métodos , fijamos un valor heurístico de $k=30$.  
- Este valor se eligió por dos motivos:  
    - Representa una reducción significativa (de 103 a 30 características), lo cual pone a prueba la capacidad de los métodos para eliminar ruido.
    - Establece una base de comparación equitativa para evaluar la calidad de las características seleccionadas por cada uno de los tres enfoques, más que la cantidad.

In [64]:
k_best_selector = SelectKBest(score_func=f_classif, k=30)
k_best_selector.fit(X_train_balanced, y_train_balanced)

feature_scores_filter = k_best_selector.scores_
features_elegidas_filter = k_best_selector.get_support()

print(f"Índices de las características elegidas: {np.where(features_elegidas_filter)}")

Índices de las características elegidas: (array([ 0,  1,  2,  3,  4,  5,  9, 10, 14, 16, 18, 19, 20, 21, 24, 25, 26,
       27, 28, 30, 35, 36, 37, 38, 39, 40, 43, 44, 71, 84]),)


Métodos wrapper:
- Elegimos RFE porque es un método Wrapper potente que sí analiza las interacciones, y usamos LogisticRegression dentro de él porque es un modelo muy rápido que nos permite ejecutar RFE en un tiempo razonable.

In [65]:
estimator = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)
rfe_selector = RFE(estimator=estimator, n_features_to_select=30, step=1)

rfe_selector.fit(X_train_balanced, y_train_balanced)

features_elegidas_wrapper = rfe_selector.support_

print(f"Índices de las características elegidas: {np.where(rfe_selector.ranking_ == 1)}")

Índices de las características elegidas: (array([ 5, 13, 16, 24, 25, 26, 28, 29, 30, 33, 35, 37, 39, 45, 48, 49, 51,
       52, 53, 58, 70, 71, 72, 73, 74, 78, 80, 83, 85, 86]),)


Métodos embebidos:
- Hemos seleccionado LASSO (Regularización L1) como nuestro método embebido porque, de las técnicas de regularización presentadas en la asignatura, es la única diseña específicamente para la selección de características. 
-  Los apuntes también mencionan Ridge (Regularización L2), pero este método solo reduce el peso de las características (usando el cuadrado de los pesos) sin forzarlas a ser exactamente cero.  
- LASSO (L1), en cambio, sí "elimina aquellas con menor valor" al forzar sus coeficientes a cero, realizando así una verdadera selección de características en un único y eficiente paso de entrenamiento.

In [66]:
lasso_model = LogisticRegression(
    penalty='l1', 
    solver='liblinear', 
    C=0.1, 
    random_state=42,
    max_iter=1000
)

embedded_selector = SelectFromModel(
    lasso_model, 
    max_features=30,
    threshold=-np.inf
) 

embedded_selector.fit(X_train_balanced, y_train_balanced)
features_elegidas_embedded = embedded_selector.get_support()

print(f"Índices de las características elegidas: {np.where(features_elegidas_embedded)}")

Índices de las características elegidas: (array([ 0,  5,  7, 11, 12, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28,
       29, 30, 34, 35, 36, 37, 38, 39, 40, 41, 45, 71, 75]),)


Usaremos LogisticRegresion para ver cuál es la mejor opción entrando y viendo el f1-score

In [67]:
model_juez = LogisticRegression(solver='liblinear', random_state=32)

features_consenso_2 = (features_elegidas_filter & features_elegidas_wrapper) | \
                      (features_elegidas_filter & features_elegidas_embedded) | \
                      (features_elegidas_wrapper & features_elegidas_embedded) # Que estén en al menos 2 de los 3 tipos
                      
features_consenso_3 = (features_elegidas_filter & features_elegidas_wrapper & features_elegidas_embedded) # Que estén en los 3

num_consenso_2 = np.sum(features_consenso_2)
num_consenso_3 = np.sum(features_consenso_3)


# Baseline (Todas las 87)
model_juez.fit(X_train_balanced, y_train_balanced)
preds_all = model_juez.predict(X_val_preprocessed)
f1_all = f1_score(y_val, preds_all)
print(f"Baseline (87 features):      {f1_all:.4f}")

# Filtro
X_train_fil = X_train_balanced[:, features_elegidas_filter]
X_val_fil = X_val_preprocessed[:, features_elegidas_filter]
model_juez.fit(X_train_fil, y_train_balanced)
preds_fil = model_juez.predict(X_val_fil)
f1_filtro = f1_score(y_val, preds_fil)
print(f"Método 'Filtro' (30 features):   {f1_filtro:.4f}")

# Wrapper
X_train_wra = X_train_balanced[:, features_elegidas_wrapper]
X_val_wra = X_val_preprocessed[:, features_elegidas_wrapper]
model_juez.fit(X_train_wra, y_train_balanced)
preds_wra = model_juez.predict(X_val_wra)
f1_wrapper = f1_score(y_val, preds_wra)
print(f"Método 'Wrapper' (30 features):  {f1_wrapper:.4f}")

# Embebido
X_train_emb = X_train_balanced[:, features_elegidas_embedded]
X_val_emb = X_val_preprocessed[:, features_elegidas_embedded]
model_juez.fit(X_train_emb, y_train_balanced)
preds_emb = model_juez.predict(X_val_emb)
f1_embebido = f1_score(y_val, preds_emb)
print(f"Método 'Embebido' (30 features): {f1_embebido:.4f}")

# En al menos 2
X_train_c2 = X_train_balanced[:, features_consenso_2]
X_val_c2 = X_val_preprocessed[:, features_consenso_2]
model_juez.fit(X_train_c2, y_train_balanced)
preds_c2 = model_juez.predict(X_val_c2)
f1_c2 = f1_score(y_val, preds_c2)
print(f"Consenso 'Al menos 2' ({num_consenso_2} features): {f1_c2:.4f}")

# En los 3
X_train_c3 = X_train_balanced[:, features_consenso_3]
X_val_c3 = X_val_preprocessed[:, features_consenso_3]
model_juez.fit(X_train_c3, y_train_balanced)
preds_c3 = model_juez.predict(X_val_c3)
f1_c3 = f1_score(y_val, preds_c3)
print(f"Consenso 'Todos 3' ({num_consenso_3} features):   {f1_c3:.4f}")

Baseline (87 features):      0.6828
Método 'Filtro' (30 features):   0.6698
Método 'Wrapper' (30 features):  0.6265
Método 'Embebido' (30 features): 0.6310
Consenso 'Al menos 2' (23 features): 0.6312
Consenso 'Todos 3' (10 features):   0.5731


Aunque el método de Filtro (30) ofreció un F1-score ligeramente superior (0.6698), se ha optado por el conjunto de 23 características ('Consenso Al menos 2'). Se ha priorizado la robustez del modelo, ya que este conjunto representa las características en las que al menos dos de los tres métodos de selección coincidieron, eliminando así características menos fiables y creando un dataset más estable para la comparativa general de modelos."

Exportamos en el formato pedido

In [68]:
X_train_final = X_train_balanced[:, features_consenso_2]
X_val_final = X_val_preprocessed[:, features_consenso_2]

feature_names = preprocessor.get_feature_names_out()
final_feature_names = feature_names[features_consenso_2]

df_train = pd.DataFrame(X_train_final.toarray(), columns=final_feature_names)
df_train['salario'] = y_train_balanced.values

df_val = pd.DataFrame(X_val_final.toarray(), columns=final_feature_names)
df_val['salario'] = y_val.values

df_processed_final = pd.concat([df_train, df_val], axis=0)
compression_opts = dict(method='zip', archive_name='processed_dataset.csv')
df_processed_final.to_csv(
    'processed_dataset.csv.zip', 
    index=False, 
    compression=compression_opts
)